<a href="https://colab.research.google.com/github/mentalMint/gpu-programming-fit/blob/main/Lab_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%writefile cuda_specs.cu

#include <iostream>
#include <cuda_runtime.h>

int main() {
    int deviceCount;
    cudaGetDeviceCount(&deviceCount); // Get the number of CUDA-capable devices

    if (deviceCount == 0) {
        std::cerr << "No CUDA devices found." << std::endl;
        return 1;
    }

    for (int i = 0; i < deviceCount; ++i) {
        cudaDeviceProp prop{}; // Initialize a cudaDeviceProp structure
        cudaGetDeviceProperties(&prop, i); // Get properties for device 'i'

        std::cout << "--- Device Number: " << i << " ---" << std::endl;
        std::cout << "  Device Name: " << prop.name << std::endl;
        std::cout << "  Compute Capability: " << prop.major << "." << prop.minor << std::endl;
        std::cout << "  Total Global Memory (bytes): " << prop.totalGlobalMem << std::endl;
        std::cout << "  Max Threads per Block: " << prop.maxThreadsPerBlock << std::endl;
        std::cout << "  Multiprocessor Count: " << prop.multiProcessorCount << std::endl;
        std::cout << "  Clock Rate (kHz): " << prop.clockRate << std::endl;
        std::cout << "  Shared Memory per Block (bytes): " << prop.sharedMemPerBlock << std::endl;
        std::cout << "  Warp Size: " << prop.warpSize << std::endl;
        std::cout << "  ECC Enabled: " << (prop.ECCEnabled ? "Yes" : "No") << std::endl;
        std::cout << std::endl;
    }

    return 0;
}

Writing cuda_specs.cu


In [2]:
!nvcc -arch=sm_75 cuda_specs.cu -o cuda_specs
!./cuda_specs

--- Device Number: 0 ---
  Device Name: Tesla T4
  Compute Capability: 7.5
  Total Global Memory (bytes): 15828320256
  Max Threads per Block: 1024
  Multiprocessor Count: 40
  Clock Rate (kHz): 1590000
  Shared Memory per Block (bytes): 49152
  Warp Size: 32
  ECC Enabled: Yes



In [3]:
%%writefile filters.cu

#include <cuda_runtime.h>
#include <png.h>
#include <iostream>
#include <vector>
#include <chrono>

#define BLOCK 32
#define R 1

//----------------------------------------------------------
// PNG LOADER
//----------------------------------------------------------
unsigned char* loadPNG(const char* filename, int& width, int& height) {
    FILE* fp = fopen(filename, "rb");
    if (!fp) {
        std::cerr << "Could not open file " << filename << "\n";
        return nullptr;
    }

    png_structp png = png_create_read_struct(PNG_LIBPNG_VER_STRING,
                                             nullptr, nullptr, nullptr);
    png_infop info = png_create_info_struct(png);

    png_init_io(png, fp);
    png_read_info(png, info);

    width  = png_get_image_width(png, info);
    height = png_get_image_height(png, info);
    int color_type = png_get_color_type(png, info);

    if (color_type != PNG_COLOR_TYPE_GRAY) {
        std::cerr << "Input must be grayscale PNG\n";
        exit(1);
    }

    png_read_update_info(png, info);

    unsigned char* data = new unsigned char[width * height];
    std::vector<png_bytep> rows(height);

    for (int i = 0; i < height; i++)
        rows[i] = data + i * width;

    png_read_image(png, rows.data());
    fclose(fp);
    return data;
}

//----------------------------------------------------------
// PNG SAVER
//----------------------------------------------------------
void savePNG(const char* filename, unsigned char* img, int w, int h)
{
    FILE* fp = fopen(filename, "wb");

    png_structp png =
        png_create_write_struct(PNG_LIBPNG_VER_STRING, nullptr, nullptr, nullptr);
    png_infop info = png_create_info_struct(png);

    png_init_io(png, fp);

    png_set_IHDR(png, info,
                 w, h,
                 8,
                 PNG_COLOR_TYPE_GRAY,
                 PNG_INTERLACE_NONE,
                 PNG_COMPRESSION_TYPE_BASE,
                 PNG_FILTER_TYPE_BASE);

    png_write_info(png, info);

    std::vector<png_bytep> rows(h);
    for (int y = 0; y < h; y++)
        rows[y] = img + y * w;

    png_write_image(png, rows.data());
    png_write_end(png, nullptr);

    fclose(fp);
}
// =======================================================
// ================     SHARED MEMORY     ================
// =======================================================

__global__
void conv3_shared(const unsigned char* in, unsigned char* out,
                  int w, int h, const float* kernel) {
    __shared__ unsigned char tile[BLOCK + 2*R][BLOCK + 2*R];

    int x = blockIdx.x * BLOCK + threadIdx.x;
    int y = blockIdx.y * BLOCK + threadIdx.y;

    int lx = threadIdx.x + R;
    int ly = threadIdx.y + R;

    // Загружаем центральный пиксель
    tile[ly][lx] = in[min(max(y, 0), h-1) * w +
                      min(max(x, 0), w-1)];

    // Загружаем пиксели границ
    // Загружаем halo (лево-право)
    if (threadIdx.x < R) {
        int gxL = x - R;
        int gxR = x + BLOCK;

        tile[ly][lx - R] = in[min(max(y, 0), h-1) * w +
                              min(max(gxL,0), w-1)];

        tile[ly][lx + BLOCK] = in[min(max(y, 0), h-1) * w +
                                  min(max(gxR,0), w-1)];
    }

    // Загружаем halo (верх-низ)
    if (threadIdx.y < R) {
        int gyT = y - R;
        int gyB = y + BLOCK;

        tile[ly - R][lx] = in[min(max(gyT,0), h-1) * w +
                              min(max(x,0), w-1)];

        tile[ly + BLOCK][lx] = in[min(max(gyB,0), h-1) * w +
                                  min(max(x,0), w-1)];
    }

    // Загружаем углы
    if (threadIdx.x < R && threadIdx.y < R) {

        int gxL = x - R;
        int gxR = x + BLOCK;
        int gyT = y - R;
        int gyB = y + BLOCK;

        // top-left
        tile[ly - R][lx - R] =
            in[min(max(gyT,0),h-1) * w + min(max(gxL,0),w-1)];

        // top-right
        tile[ly - R][lx + BLOCK] =
            in[min(max(gyT,0),h-1) * w + min(max(gxR,0),w-1)];

        // bottom-left
        tile[ly + BLOCK][lx - R] =
            in[min(max(gyB,0),h-1) * w + min(max(gxL,0),w-1)];

        // bottom-right
        tile[ly + BLOCK][lx + BLOCK] =
            in[min(max(gyB,0),h-1) * w + min(max(gxR,0),w-1)];
    }

    __syncthreads();

    if(x >= w || y >= h) return;

    float sum = 0;

    for(int dy = -R; dy <= R; dy++)
        for(int dx = -R; dx <= R; dx++)
            sum += kernel[(dy+1)*3 + (dx+1)] *
                   tile[ly + dy][lx + dx];

    out[y * w + x] = (unsigned char)min(max(sum, 0.0f), 255.0f);
}


// =======================================================
// ===================  TEXTURE MEMORY ===================
// =======================================================

__global__
void conv3_tex(unsigned char* out, int w, int h,
               cudaTextureObject_t tex, const float* kernel)
{
    int x = blockIdx.x * BLOCK + threadIdx.x;
    int y = blockIdx.y * BLOCK + threadIdx.y;

    if(x >= w || y >= h) return;

    float sum = 0;

    for(int ky = -1; ky <= 1; ky++)
        for(int kx = -1; kx <= 1; kx++){
            float xx = min(max(x + kx, 0), w - 1);
            float yy = min(max(y + ky, 0), h - 1);

            unsigned char v = tex2D<unsigned char>(tex, xx, yy);
            sum += kernel[(ky + 1) * 3 + (kx + 1)] * v;
        }

    out[y * w + x] = (unsigned char) min(max(sum, 0.0f), 255.0f);
}


int main(int argc, char* argv[]) {
    if (argc < 2) {
        std::cerr << "Usage: " << argv[0] << " <N>" << std::endl;
        return 1; // Indicate an error
    }

    int N = 0;
    try {
        N = std::stoi(argv[1]); // Convert the first argument to an integer
        std::cout << "Iterations number: " << N << std::endl;
    } catch (const std::invalid_argument& e) {
        std::cerr << "Invalid argument: " << e.what() << std::endl;
        return 1;
    } catch (const std::out_of_range& e) {
        std::cerr << "Number out of range: " << e.what() << std::endl;
        return 1;
    }

    int W, H;
    unsigned char* h_in = loadPNG("input.png", W, H);

    size_t size = W * H;
    unsigned char* h_out = new unsigned char[size];

    // Blur
/*

    float h_kernel[9] = {
        1/9.f, 1/9.f, 1/9.f,
        1/9.f, 1/9.f, 1/9.f,
        1/9.f, 1/9.f, 1/9.f
    };
*/

    // Sobel
    float h_kernel[9] = {
        -1.f, 0.f, 1.f,
        -2.f, 0.f, 2.f,
        -1.f, 0.f, 1.f
    };

    float* d_kernel;
    cudaMalloc(&d_kernel, 9*sizeof(float));
    cudaMemcpy(d_kernel, h_kernel, 9*sizeof(float), cudaMemcpyHostToDevice);

    unsigned char *d_in, *d_out;
    cudaMalloc(&d_in, size);
    cudaMalloc(&d_out, size);
    cudaMemcpy(d_in, h_in, size, cudaMemcpyHostToDevice);

    dim3 block(BLOCK, BLOCK);
    dim3 grid((W + BLOCK - 1) / BLOCK, (H + BLOCK - 1) / BLOCK);

    // -------------------------------
    // SHARED MEMORY
    // -------------------------------

    auto time = 0;

    for (int i = 0; i <= N + 10; i++) {
        auto t1 = std::chrono::high_resolution_clock::now();
        conv3_shared<<<grid, block, BLOCK * sizeof(unsigned char)>>>(d_in, d_out, W, H, d_kernel);
        cudaDeviceSynchronize();
        auto t2 = std::chrono::high_resolution_clock::now();
        cudaMemcpy(h_out, d_out, size, cudaMemcpyDeviceToHost);
        if (i <= 10) {
            continue;
        }
        time += std::chrono::duration_cast<std::chrono::nanoseconds>(t2-t1).count();
    }

    std::cout << "Shared memory time (avg): "
        << time / N
        << " ns\n";

    cudaMemcpy(h_out, d_out, size, cudaMemcpyDeviceToHost);
    savePNG("output1.png", h_out, W, H);

    // -------------------------------
    // TEXTURE MEMORY
    // -------------------------------
    cudaArray* arr;
    cudaChannelFormatDesc desc =
        cudaCreateChannelDesc<unsigned char>();

    cudaMallocArray(&arr, &desc, W, H);

    cudaMemcpy2DToArray(arr, 0, 0,
                        h_in, W*sizeof(unsigned char),
                        W*sizeof(unsigned char), H,
                        cudaMemcpyHostToDevice);

    cudaResourceDesc res = {};
    res.resType = cudaResourceTypeArray;
    res.res.array.array = arr;

    cudaTextureDesc texD = {};
    texD.addressMode[0] = cudaAddressModeClamp;
    texD.addressMode[1] = cudaAddressModeClamp;
    texD.filterMode = cudaFilterModePoint;
    texD.readMode = cudaReadModeElementType;

    cudaTextureObject_t tex = 0;
    cudaCreateTextureObject(&tex, &res, &texD, nullptr);

    auto time2 = 0;

    for (int i = 0; i <= N + 10; i++) {
        auto t3 = std::chrono::high_resolution_clock::now();
        conv3_tex<<<grid, block>>>(d_out, W, H, tex, d_kernel);
        cudaDeviceSynchronize();
        auto t4 = std::chrono::high_resolution_clock::now();
        cudaMemcpy(h_out, d_out, size, cudaMemcpyDeviceToHost);
        if (i <= 10) {
            continue;
        }
        time2 += std::chrono::duration_cast<std::chrono::nanoseconds>(t4-t3).count();
    }

    std::cout << "Texture memory time (avg): "
        << time2 / N
        << " ns\n";

    savePNG("output2.png", h_out, W, H);

    // cleanup
    cudaDestroyTextureObject(tex);
    cudaFreeArray(arr);
    cudaFree(d_in);
    cudaFree(d_out);
    cudaFree(d_kernel);

    delete[] h_in;
    delete[] h_out;

    return 0;
}


Writing filters.cu


In [6]:
!nvcc -arch=sm_75 --use_fast_math filters.cu -o filters -lpng -lz
!./filters 10000

Iterations number: 10000
Shared memory time (avg): 47764 ns
Texture memory time (avg): 40803 ns
